In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import doctest
from sklearn.metrics import accuracy_score
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df_train=pd.read_csv("/kaggle/input/mnist-in-csv/mnist_train.csv")
df_test=pd.read_csv("/kaggle/input/mnist-in-csv/mnist_test.csv")
#выбираем 2 цифры на любой вкус
num1=0
num2=1
filter1=df_train["label"]==num1
filter2=df_train["label"]==num2
filter3=df_test["label"]==num1
filter4=df_test["label"]==num2
data_train=df_train.where(filter1 | filter2)
data_test=df_train.where(filter3 | filter4)
data_train=data_train.dropna()
data_test=data_train.dropna()
print(data_train.shape)
data_train

In [ ]:
data_train1 = pd.get_dummies(data_train,columns=['label'],drop_first=True)
data_train1.rename(columns={data_train1.columns.tolist()[-1]: 'label'}, inplace=True)

data_test1 = pd.get_dummies(data_test,columns=['label'],drop_first=True)
data_test1.rename(columns={data_test1.columns.tolist()[-1]: 'label'}, inplace=True)

data_test1

In [ ]:
print(data_train.shape)

data_train

In [ ]:
X_train=data_train1.drop('label',axis=1)
X_train = X_train /255
Y_train=data_train1['label']

X_test=data_test1.drop('label',axis=1)
X_test = X_test /255
Y_test=data_test1['label']

X_train1=X_train.to_numpy()
X_train1=np.resize(X_train,(len(X_train),28,28))

X_test1=X_test.to_numpy()
X_test1=np.resize(X_test,(len(X_test),28,28))

Y_test1=Y_test.to_numpy()

images=5
fig, ax = plt.subplots(nrows=1, ncols=images)
for i in range(images):
    ax[i].imshow(X_train1[i])
plt.show()

In [ ]:
#переводим пиксели, которые закрашены хоть как-то, в закрашены полностью
X_train1=X_train1.astype(bool).astype(int)
X_test1=X_test1.astype(bool).astype(int)
Y_test1=Y_test1.astype(bool).astype(int)

images=6
fig, ax = plt.subplots(nrows=1, ncols=images)
for i in range(images):
    ax[i].imshow(X_train1[i])
plt.show()

In [ ]:
class EMAlgorithm(object):
    
    def __init__(self, iterations=10):     
        self.iterations = iterations   
      
    def ptxk(self, x,prob):
        """
        Return the probability that the picture x shows a number k
    
        >>> EM = EMAlgorithm()
        >>> EM.ptxk(np.array([[[0,1],[0,1]],[[1,0],[1,0]]]), np.array([[0.5, 0.5],[0.5, 0.5]]))
        array([[[0.5, 0.5],
                [0.5, 0.5]],
        <BLANKLINE>
               [[0.5, 0.5],
                [0.5, 0.5]]])
        >>> EM.ptxk(np.array([[[1,0],[1,0]],[[0,1],[0,1]]]), np.array([[0.25, 0.75],[0.75, 0.25]]))
        array([[[0.25, 0.75],
                [0.25, 0.75]],
        <BLANKLINE>
               [[0.75, 0.25],
                [0.75, 0.25]]])
        >>> EM.ptxk(np.array([[[0,0],[0,0]],[[0,0],[0,0]]]), np.array([[0, 1],[1, 0]]))
        array([[[0., 0.],
                [0., 0.]],
        <BLANKLINE>
               [[0., 0.],
                [0., 0.]]])
        """
        #Сумма pt(k|x) по всем X
        summa = np.sum(prob,axis=0) 
        ptx_k = np.zeros((2, x.shape[1], x.shape[2]))
    
        #Сумма xi * pt(k|x)
        for i in range(prob.shape[0]):
            ptx_k[0] += x[i]*prob[i][0]
            ptx_k[1] += x[i]*prob[i][1]
    
        #Сумма xi * pt(k|x) / Сумма pt(k|x)
        ptx_k[0]=ptx_k[0]/summa[0]
        ptx_k[1]=ptx_k[1]/summa[1]
    
        return ptx_k
                     
                     

    def ptkx(self, ptxk, pk):
        """
        Return the probability that the picture x belongs to the class k
    
        >>> EM = EMAlgorithm()
        >>> EM.ptkx(np.array([[0.25, 0.75],[0.5, 0.5]]), np.array([0.45, 0.55]))
        array([[0.21428571, 0.78571429],
               [0.45      , 0.55      ]])
        """
        summa = (ptxk[:,0]*pk[0] + ptxk[:,1]*pk[1])
        #Сумма pt(x|0)
        pt0x = (ptxk[:,0] * pk[0])/summa
        #Сумма pt(x|1)
        pt1x = (ptxk[:,1]*pk[1])/summa
        return np.stack((pt0x, pt1x), axis=-1)

    def ptk(self, aposterior_probs):
        """
        Return average for each column
    
        >>> EM = EMAlgorithm()
        >>> EM.ptk(np.array([[0,1],[0.5,0],[0.5,0],[1,1]]))
        array([0.5, 0.5])
        >>> EM.ptk(np.array([[0.,0.]]))
        array([0., 0.])
        """
        return aposterior_probs.mean(axis=0)
    
    def bern_distr(self, x, params):
        """
        Return Bernoulli Distribution of x with parametres from params
        
        >>> EM = EMAlgorithm()
        >>> EM.bern_distr(np.array([[0,0],[0,1]]), np.array([0.5, 1]))
        array([[0.25, 0.  ],
               [0.25, 0.  ]])
        """
        
        params = np.repeat(np.expand_dims(params, axis=0), repeats=x.shape[0], axis=0)
        params = params.reshape((params.shape[0], params.shape[1], -1))
        x = np.repeat(np.expand_dims(x, axis=1), repeats=2, axis=1)
        x = x.reshape((x.shape[0], x.shape[1], -1))
        return np.multiply(np.power(params, x).prod(axis=-1), np.power(1 - params, (1 - x)).prod(axis=-1))
    
    def p0(self ,n):
        x = np.random.random((n, 1))
        return np.concatenate((x, 1-x),axis=1)
    
    def fit(self, X_train, X_test, Y_test):
        pkx = self.p0(X_train.shape[0])
        for iteration in range(self.iterations):
            print("ITER",iteration + 1)
            pk=self.ptk(pkx)
            pxk = self.ptxk(X_train, pkx)
            pkx = self.ptkx(self.bern_distr(X_train,pxk), pk)   
            prediction = self.bern_distr(X_test, pxk).argmax(axis=1)
            print("accuracy", accuracy_score(prediction,Y_test)) 
            fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2)
            ax1.imshow(pxk[0])
            ax2.imshow(pxk[1])
        
        print(doctest.testmod())

In [ ]:
EM = EMAlgorithm(iterations = 5)
EM.fit(X_train1, X_test1, Y_test1)